<a href="https://colab.research.google.com/github/sadafyousuf/Spoon-Knife/blob/master/Flowers_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import glob
 
import numpy as np
import PIL #reading images
import PIL.Image #display examples
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
camomile_images = '/kaggle/input/flowers-recognition/flowers/camomile/*jpg'
dendelion_images = '/kaggle/input/flowers-recognition/flowers/dandelion/*jpg'
rose_images = '/kaggle/input/flowers-recognition/flowers/rose/*jpg'
sunflower_images = '/kaggle/input/flowers-recognition/flowers/sunflower/*jpg'
tulip_images = '/kaggle/input/flowers-recognition/flowers/tulip/*jpg'
list_of_flowers = [camomile_images, dendelion_images,rose_images, sunflower_images, tulip_images]


In [ ]:
PIL.Image.open(glob.glob(tulip_images)[0])


In [ ]:
batch_size = 64
img_height = 200
img_width = 200

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  "/kaggle/input/flowers-recognition/flowers",
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
validation_ds = tf.keras.preprocessing.image_dataset_from_directory(
  "/kaggle/input/flowers-recognition/flowers",
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


In [ ]:
class_names = train_ds.class_names
plt.figure(figsize=(15, 15));


In [ ]:
for img, label in train_ds.take(1):
    for i in range(12):
        ax = plt.subplot(3,4, 1 + i)
        plt.imshow(img[i].numpy().astype('uint8'))
        plt.title(class_names[label[i]])
        plt.axis("off")


In [ ]:
from tensorflow.keras import layers

normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)


In [ ]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = validation_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
num_classes = 6
model = tf.keras.Sequential([
    layers.experimental.preprocessing.Rescaling(1./255),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, activation='linear'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes)
])


In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])


In [ ]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=5
)
